In [14]:
import pandas as pd
import minsearch as ms
from openai import OpenAI

In [2]:
# Cargamos los datos
df = pd.read_csv('../DATASETS/faq_sacmex.csv')

In [4]:
df['document'] = 'faq_sacmex'

In [5]:
index = ms.Index(
    text_fields=['pregunta', 'respuesta'],
    keyword_fields=['document']
)

In [6]:
documents = df.to_dict(orient='records')
documents

[{'id': 'd7c4ce5eda85cd602edc71a3f29193e0',
  'pregunta': '¿Dónde puedo reportar una fuga de agua?',
  'respuesta': 'Debes reportarla al organismo operador de agua potable y alcantarillado de tu localidad.',
  'document': 'faq_sacmex'},
 {'id': 'c6b1388a0227cae6d6045a1dc7dd82c5',
  'pregunta': '¿Qué debo hacer si veo una fuga de agua en la calle?',
  'respuesta': 'En la Ciudad de México, debes reportarla al 55 5654 3210 para que una brigada del Sistema de Aguas de la Ciudad de México acuda a repararla.',
  'document': 'faq_sacmex'},
 {'id': 'b9e493259e0b9fb9f94e4f4d66ab8ded',
  'pregunta': '¿Qué debo hacer si veo una fuga de agua en mi casa?',
  'respuesta': 'Lo primero que debes hacer es cortar el suministro de agua, ya sea con el grifo de cierre o la válvula de cierre, que generalmente se encuentran en el baño o la cocina.\xa0',
  'document': 'faq_sacmex'},
 {'id': '5229ec6562607f2de830a7826a099964',
  'pregunta': '¿Qué debo hacer si mi consumo de agua parece haberse elevado?',
  're

In [7]:
for doc in documents:
    for key in doc:
        if isinstance(doc[key], float):
            doc[key] = str(doc[key])

index.fit(documents)

In [8]:
def search(query):
    boost = {'pregunta': 3.0, 'respuesta': 0.5}
    # boost = {}
    number_of_results = 5

    results = index.search(
        query,
        filter_dict={},
        boost_dict=boost,
        num_results=number_of_results
    )
    return results

In [9]:
documents[0]

{'id': 'd7c4ce5eda85cd602edc71a3f29193e0',
 'pregunta': '¿Dónde puedo reportar una fuga de agua?',
 'respuesta': 'Debes reportarla al organismo operador de agua potable y alcantarillado de tu localidad.',
 'document': 'faq_sacmex'}

In [ ]:
entry_template = """
pregunta: {pregunta}
respuesta: {respuesta}
""".strip()

prompt_template = """
Eres un agente chatbot que ayuda a levantar reportes de fugas de agua en la SACMEX. Responde a la PREGUNTA de abajo basado en el CONTEXTO. Usa el CONTEXTO para dar una repuesta precisa.

PREGUNTA: {pregunta}

CONTEXTO: {respuesta}
"""


def build_prompt(query, results):
    context = ""
    for doc in results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(pregunta=query, respuesta=context).strip()
    return prompt

In [11]:
query = documents[0]['pregunta']

In [13]:
search_results = search(query)
prompt = build_prompt(query, search_results)
print(prompt)

Eres un agente chatbot que ayuda a levantar reportes de fugas de agua en la SACMEX. Responde a la PREGUNTA de abajo basado en el CONTEXTO. Usa el CONTEXTO para dar una repuesta para dar una respuesta precisa.

PREGUNTA: ¿Dónde puedo reportar una fuga de agua?

CONTEXTO: pregunta: ¿Dónde puedo reportar una fuga de agua?
respuesta: Para reportar una fuga de agua en la Ciudad de México, puedes: 
Llamar al LOCATEL al *0311 o al 55 5658 1111 
Contactar a las redes sociales oficiales del SACMEX en X (antes Twitter) y @SistemaDeAguasCDMX en Facebook 
Ingresar a la página web de atención ciudadana de la CDMX y llenar el cuadro con la descripción del servicio que requieres 
Si el daño es imputable al usuario, deberá cubrir el costo de la reparación. Para ello, se puede acudir al Área de Atención Ciudadana más cercana al domicilio.

pregunta: ¿Dónde puedo reportar una fuga de agua?
respuesta: Debes reportarla al organismo operador de agua potable y alcantarillado de tu localidad.

pregunta: ¿Qué

In [ ]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [ ]:
def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                'role': 'system',
                'content': 'Eres un agente útil.'
            },
            {
                'role': 'user',
                'content': prompt
            }
        ]
    )
    return response.choices[0].message.content

In [ ]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt, 'llama3.2:1b')
    return answer

In [ ]:
query = 'Whats a good chinese meal that uses soy sauce and is low fat.'
answer = rag(query)
print(answer)